In [3]:
print("Hello World!")

Hello World!


In [ ]:
import mysql.connector
# Connect to MySQL server
mydb = mysql.connector.connect(
  host="localhost",
  user="tomek",
  password="tomek123"
)

# Create Database
mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE mydatabase")

In [6]:
# Show databases
mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

('information_schema',)
('mydatabase',)
('mysql',)
('performance_schema',)
('sys',)


In [26]:
import pandas as pd
# Load data_categories as dataframe
data_categories = pd.read_csv('.\\data\\raw\\data_categories.csv', index_col=False, delimiter = ';')

# Load data_market_prices as dataframe
data_market_prices = pd.read_csv('.\\data\\raw\\data_market_prices.csv', index_col=False, low_memory=False, delimiter = ',')

# Load data_vas_purchases as dataframe
data_vas_purchases = pd.read_csv('.\\data\\raw\\data_vas_purchases.csv', index_col=False, delimiter = ';')

#data_categories.head()
#print(data_categories)

#data_market_prices.head()
#print(data_market_prices)

#data_vas_purchases.head()
#print(data_vas_purchases)

data = pd.merge(data_market_prices,data_categories,on='category_id',how='left')

data_full = pd.merge(data,data_vas_purchases,on='ad_id',how='left')

data_full.head()



,ad_id,city_id_x,category_id_x,market,date_posted,date_expired,price_per_sqm,price,rooms_num,category_name,subcategory_name,date_day,city_id_y,category_id_y,amount_spend_to_promote_ads
0,62763753,26,102,NaN,2/12/2022,3/14/2022,57.0,2600.0,2,Rent,Apartments,NaN,NaN,NaN,NaN
1,62888959,26,102,NaN,3/17/2022,3/17/2022,47.0,9000.0,5,Rent,Apartments,NaN,NaN,NaN,NaN
2,62925670,26,101,primary,3/28/2022,5/12/2022,15968.0,1390000.0,3,Sale,Apartments,NaN,NaN,NaN,NaN
3,62261719,1004,101,secondary,9/9/2021,3/4/2022,6743.0,499000.0,3,Sale,Apartments,NaN,NaN,NaN,NaN
4,62878680,26,102,NaN,3/15/2022,3/28/2022,57.0,2500.0,2,Rent,Apartments,NaN,NaN,NaN,NaN


In [32]:
data_drop_na = data_full.dropna()
data_drop_na.head()
data_drop_na.shape

(3434, 15)

In [22]:
data_categories.head()
#print(data_categories)


,category_id,category_name,subcategory_name
0,11,Investments,Investments
1,302,Rent,Rooms
2,402,Rent,Plots
3,502,Rent,Commercials
4,101,Sale,Apartments


In [23]:
data_market_prices.head()
#print(data_market_prices)


,ad_id,city_id,category_id,market,date_posted,date_expired,price_per_sqm,price,rooms_num
0,62763753,26,102,NaN,2/12/2022,3/14/2022,57.0,2600.0,2
1,62888959,26,102,NaN,3/17/2022,3/17/2022,47.0,9000.0,5
2,62925670,26,101,primary,3/28/2022,5/12/2022,15968.0,1390000.0,3
3,62261719,1004,101,secondary,9/9/2021,3/4/2022,6743.0,499000.0,3
4,62878680,26,102,NaN,3/15/2022,3/28/2022,57.0,2500.0,2


In [24]:
data_vas_purchases.head()
#print(data_vas_purchases)

,date_day,ad_id,city_id,category_id,amount_spend_to_promote_ads
0,2022-03-06,62845828,26,102,22.19
1,2021-03-03,61574280,26,101,11.09
2,2021-03-23,61540306,26,102,22.19
3,2022-02-26,62819726,26,102,22.19
4,2022-03-11,62679776,26,101,18.86


In [ ]:
#NOTATKI
# Drop new formatted date_posted and date_expired columns
#clean_data_market_price =  clean_data_market_price.drop(columns=['date_posted', 'date_expired'])


# Check real estate ads distribution based on the day (from date_posted column)
mycursor.execute("""
					{CTE}

					          SELECT DATE_FORMAT(date_posted, '%Y/%m/%d'), COUNT(*) as c
					          FROM clean_data_market_prices 
                    WHERE DATE_FORMAT(date_posted, '%Y') = 2022
                    GROUP BY YEAR(date_posted), MONTH(date_posted), DAY(date_posted)
                    ORDER BY c DESC;
				""".format(CTE=clean_data_market_price_CTE))

# Get output
myresult = mycursor.fetchall()

# Print output
for x in myresult:
  print(x)

# OUTPUT COMMENT
# Generaly speaking for 2021 more offers were posted in the middle of the month (February, days: 10-19) whereas
# for 2022 its the end of the March (days: )




mycursor.execute("""
					{CTE}

					          SELECT DATE_FORMAT(date_expired, '%Y/%m'), COUNT(*)
					          FROM clean_data_market_prices 
                    WHERE DATE_FORMAT(date_expired, '%Y') = 2021
                      AND (DATE_FORMAT(date_posted, '%m/%Y') = DATE_FORMAT(date_expired, '%m/%Y'))
                    GROUP BY YEAR(date_expired), MONTH(date_expired);
				""".format(CTE=clean_data_market_price_CTE))